In [7]:
# %load CNN_MNIST_pytorch.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import random

# 設置seed
seed = 198964
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Dataloader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, num_workers = 3)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True,num_workers = 3)

增加兩行convolution layers，並增加neuron數量

In [8]:
#Define Network, we implement LeNet here
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5),stride=1, padding=0)
        self.conv2 = nn.Conv2d(6, 6, kernel_size=(3,3),stride=1, padding=1) #add two convolution layers
        self.conv3 = nn.Conv2d(6, 6, kernel_size=(3,3),stride=1, padding=1)
        self.conv4 = nn.Conv2d(6, 16, kernel_size=(5,5),stride=1, padding=0)
        self.fc1 = nn.Linear(16*4*4, 240) # make neuron 2x wider
        self.fc2 = nn.Linear(240, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
#         out = F.max_pool2d(out, 2) <- no need to max pool because input size isn't big enough
        out = F.relu(self.conv3(out))
#         out = F.max_pool2d(out, 2)
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1) #flatten
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

建model並設置優化器

In [9]:
model = Net() # build model
if args.cuda:
    device = torch.device('cuda')
    model.to(device)

#define optimizer/loss function
Loss = nn.CrossEntropyLoss()

# select your optimizer, here we choose RMSprop because the original code performs badly in RMSprop, so I try to fix it.

# optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
optimizer = optim.RMSprop(model.parameters(), lr=0.1, momentum=0, alpha=0.5) # Learning rate here will be replaced by next function. By the way, alpha is set to 0.99 defaultly, and I set it to 0.5 . I don't want any momentum as well.

設定learning rate的排程，使他在多個epoch之後逐漸降低

In [10]:
#learning rate scheduling
def adjust_learning_rate(optimizer, epoch):
    if epoch < 5: # After 5 epochs, everage loss always decrease slowly, so we can use smaller learning rate to avoid too much change
        lr = 0.005  # 0.01 is too big so let's try 0.005
    elif epoch < 15:
        lr = 0.0005 
    else: 
        lr = 0.00005

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [11]:
#training function
def train(epoch):
    model.train()
    adjust_learning_rate(optimizer, epoch)

    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))


#Testing function
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if args.cuda:
            data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
            test_loss += Loss(output, target).item()
            pred = output.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#run and save model
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)


Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.112505
RMSprop (
Parameter Group 0
    alpha: 0.5
    centered: False
    eps: 1e-08
    lr: 0.005
    momentum: 0
    weight_decay: 0
)

Test set: Average loss: 0.0680, Accuracy: 9791/10000 (98%)

Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.038866
RMSprop (
Parameter Group 0
    alpha: 0.5
    centered: False
    eps: 1e-08
    lr: 0.005
    momentum: 0
    weight_decay: 0
)

Test set: Average loss: 0.0987, Accuracy: 9763/10000 (98%)

Train Epoch: 3 [44928/60000 (100%)]	Loss: 0.012791
RMSprop (
Parameter Group 0
    alpha: 0.5
    centered: False
    eps: 1e-08
    lr: 0.005
    momentum: 0
    weight_decay: 0
)

Test set: Average loss: 0.0829, Accuracy: 9826/10000 (98%)

Train Epoch: 4 [44928/60000 (100%)]	Loss: 0.300899
RMSprop (
Parameter Group 0
    alpha: 0.5
    centered: False
    eps: 1e-08
    lr: 0.005
    momentum: 0
    weight_decay: 0
)

Test set: Average loss: 0.1083, Accuracy: 9725/10000 (97%)

Train Epoch: 5 [44928/60000 (100